In [ ]:
!git clone https://github.com/AnalyticsKnight/yemoonsaBigdata

Cloning into 'yemoonsaBigdata'...
remote: Enumerating objects: 672, done.
remote: Counting objects: 100% (309/309), done.
remote: Compressing objects: 100% (198/198), done.
remote: Total 672 (delta 142), reused 212 (delta 111), pack-reused 363
Receiving objects: 100% (672/672), 11.12 MiB | 17.88 MiB/s, done.
Resolving deltas: 100% (327/327), done.


# 실습 1. Motor Trend Car Road Tests

## 데이터 수집

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
X_train = pd.read_csv("/content/yemoonsaBigdata/datasets/Part2/mpg_X_train.csv")
y_train = pd.read_csv("/content/yemoonsaBigdata/datasets/Part2/mpg_y_train.csv")
X_test = pd.read_csv("/content/yemoonsaBigdata/datasets/Part2/mpg_X_test.csv")

## 데이터 전처리

In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278 entries, 0 to 277
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          278 non-null    object 
 1   mpg           278 non-null    float64
 2   cylinders     278 non-null    int64  
 3   displacement  278 non-null    float64
 4   horsepower    274 non-null    float64
 5   weight        278 non-null    int64  
 6   acceleration  278 non-null    float64
 7   model_year    278 non-null    int64  
dtypes: float64(4), int64(3), object(1)
memory usage: 17.5+ KB


In [ ]:
# 결측치 대체 (mean)
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
X_train[['horsepower']] = imputer.fit_transform(X_train[['horsepower']])
X_test[['horsepower']] = imputer.fit_transform(X_test[['horsepower']])

In [ ]:
# 변수 형태별 리스트 저장
COL_DEL = ['name']
COL_NUM = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model_year']
COL_CAT = []
COL_Y = ['isUSA']

X_train = X_train.iloc[:, 1:]
X_test = X_test.iloc[:, 1:]

## 모델링

In [ ]:
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.3)

In [ ]:
# scaling (수치형에 한해서만), validation 제외한 순수한 training data로만 scaler를 fitting 해야 함.
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_tr[COL_NUM])
X_tr[COL_NUM] = scaler.transform(X_tr[COL_NUM])
X_val[COL_NUM] = scaler.transform(X_val[COL_NUM])
X_test[COL_NUM] = scaler.transform(X_test[COL_NUM])

In [ ]:
# KNN
from sklearn.neighbors import KNeighborsClassifier

modelKNN = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
modelKNN.fit(X_tr, y_tr.values.ravel())

KNeighborsClassifier(metric='euclidean')

In [ ]:
# decision tree
from sklearn.tree import DecisionTreeClassifier

modelDT = DecisionTreeClassifier(max_depth=10)
modelDT.fit(X_tr, y_tr)

DecisionTreeClassifier(max_depth=10)

## 평가

In [ ]:
y_val_pred_probaKNN = modelKNN.predict_proba(X_val)
y_val_pred_probaDT = modelDT.predict_proba(X_val)

In [ ]:
from sklearn.metrics import roc_auc_score

scoreKNN = roc_auc_score(y_val, y_val_pred_probaKNN[:, 1])
scoreDT = roc_auc_score(y_val, y_val_pred_probaDT[:, 1])

print(scoreKNN, scoreDT)

0.8999999999999999 0.876470588235294


## 예측

In [ ]:
pred_KNN = modelKNN.predict_proba(X_test)[:, 1]
print('KNN model:', pred_KNN)
print()
pred_DT = modelDT.predict_proba(X_test)[:, 1]
print('decision tree model:', pred_DT)

KNN model: [0.6 0.8 1.  1.  0.4 0.4 0.2 0.6 0.6 0.6 1.  0.4 1.  0.6 0.2 0.2 1.  1.
 0.2 1.  1.  1.  1.  1.  1.  0.4 0.  1.  1.  0.4 0.2 0.  0.6 1.  1.  0.8
 0.4 1.  0.2 0.6 1.  0.2 0.2 0.6 0.2 0.6 0.4 1.  0.  1.  0.  0.2 1.  0.2
 1.  1.  1.  0.8 1.  0.4 1.  0.8 1.  1.  1.  1.  1.  0.2 0.8 0.2 0.8 1.
 0.2 0.  1.  0.4 1.  0.2 0.4 1.  1.  1.  0.2 1.  1.  1.  0.  0.2 0.2 1.
 1.  1.  1.  1.  1.  1.  0.  1.  1.  1.  1.  1.  1.  1.  1.  0.2 0.6 0.8
 0.4 1.  0.4 1.  0.2 1.  1.  0.  1.  1.  1.  0. ]

decision tree model: [0. 1. 1. 1. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 1. 1. 1. 1. 1.
 1. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 1. 0. 1. 0. 1.
 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1.
 1. 0. 1. 0. 1. 0. 0. 1. 1. 1. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1.
 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 0. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1. 0.]


In [ ]:
pd.DataFrame({'isUSA': pred_KNN}).to_csv('123.csv', index=False)
pd.DataFrame({'isUSA': pred_DT}).to_csv('456.csv', index=False)

# 실습 2. Penguins

## 데이터 수집

In [ ]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')

In [ ]:
X_train = pd.read_csv("/content/yemoonsaBigdata/datasets/Part2/penguin_X_train.csv")
y_train = pd.read_csv("/content/yemoonsaBigdata/datasets/Part2/penguin_y_train.csv")
X_test = pd.read_csv("/content/yemoonsaBigdata/datasets/Part2/penguin_X_test.csv")

## 데이터 전처리

In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            240 non-null    object 
 1   island             240 non-null    object 
 2   sex                232 non-null    object 
 3   bill_length_mm     238 non-null    float64
 4   bill_depth_mm      238 non-null    float64
 5   flipper_length_mm  238 non-null    float64
dtypes: float64(3), object(3)
memory usage: 11.4+ KB


In [ ]:
train = pd.concat([X_train, y_train], axis=1)
train.loc[(train.sex.isna()) | (train.bill_length_mm.isna()) | (train.bill_depth_mm.isna()) | (train.flipper_length_mm.isna()) | (train.body_mass_g.isna())]

,species,island,sex,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
0,Adelie,Torgersen,NaN,42.0,20.2,190.0,4250.0
6,Gentoo,Biscoe,NaN,44.5,14.3,216.0,4100.0
43,Gentoo,Biscoe,NaN,NaN,NaN,NaN,NaN
66,Adelie,Torgersen,NaN,37.8,17.3,180.0,3700.0
88,Gentoo,Biscoe,NaN,47.3,13.8,216.0,4725.0
89,Adelie,Torgersen,NaN,37.8,17.1,186.0,3300.0
110,Gentoo,Biscoe,NaN,44.5,15.7,217.0,4875.0
229,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN


In [ ]:
# 결측치 제거
train = train.dropna()
train.reset_index(drop=True, inplace=True)

X_train = train[['species', 'island', 'sex', 'bill_length_mm', 'bill_depth_mm', 'flipper_length_mm']]
y_train = train[['body_mass_g']]

In [ ]:
# 변수 형태별 리스트 저장
COL_DEL = []
COL_NUM = ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm']
COL_CAT = ['species', 'island', 'sex']
COL_Y = ['body_mass_g']

In [ ]:
# label incoding (범주형에 한해서만), training data 뿐만 아니라 test data에도 같이 적용

X = pd.concat([X_train, X_test])

from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(handle_unknown = 'ignore')
ohe.fit(X[COL_CAT])

X_train_res = ohe.transform(X_train[COL_CAT])
X_test_res = ohe.transform(X_test[COL_CAT])

X_train_ohe = pd.DataFrame(X_train_res.todense(), columns = ohe.get_feature_names_out())
X_test_ohe = pd.DataFrame(X_test_res.todense(), columns = ohe.get_feature_names_out())

X_train_fin = pd.concat([X_train[COL_NUM], X_train_ohe], axis=1)
X_test_fin = pd.concat([X_test[COL_NUM], X_test_ohe], axis=1)

X_train_fin

,bill_length_mm,bill_depth_mm,flipper_length_mm,species_Adelie,species_Chinstrap,species_Gentoo,island_Biscoe,island_Dream,island_Torgersen,sex_FEMALE,sex_MALE
0,43.5,15.2,213.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
1,42.8,18.5,195.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
2,53.5,19.9,205.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
3,50.2,14.3,218.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
4,36.5,18.0,182.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
227,46.6,17.8,193.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
228,49.8,15.9,229.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
229,34.6,17.2,189.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
230,45.9,17.1,190.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0


## 모델링

In [ ]:
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(X_train_fin, y_train, test_size=0.3)

In [ ]:
# scaling (수치형에 한해서만), validation 제외한 순수한 training data로만 scaler를 fitting 해야 함.
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X_tr[COL_NUM])
X_tr[COL_NUM] = scaler.transform(X_tr[COL_NUM])
X_val[COL_NUM] = scaler.transform(X_val[COL_NUM])
X_test_fin[COL_NUM] = scaler.transform(X_test_fin[COL_NUM])

In [ ]:
# linear regression
from sklearn.linear_model import LinearRegression

modelLR = LinearRegression()
modelLR.fit(X_tr, y_tr)

LinearRegression()

In [ ]:
coef = pd.Series(data=modelLR.coef_[0], index=X_train_fin.columns)
coef.sort_values()

species_Chinstrap   -502.012211
species_Adelie      -269.741866
sex_FEMALE          -224.590072
island_Dream         -50.204567
island_Torgersen      23.604181
island_Biscoe         26.600387
sex_MALE             224.590072
bill_depth_mm        495.138306
bill_length_mm       577.040738
flipper_length_mm    710.133051
species_Gentoo       771.754077
dtype: float64

## 평가

In [ ]:
y_val_pred = modelLR.predict(X_val)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_val, y_val_pred)
rmse = mean_squared_error(y_val, y_val_pred, squared=False)

print('MSE : {0:.3f} , RMSE : {1:.3F}'.format(mse, rmse))

MSE : 82610.030 , RMSE : 287.420


## 예측

In [ ]:
y_pred = modelLR.predict(X_test_fin)
y_pred

array([[4191.92908452],
       [5372.9174279 ],
       [4116.67916481],
       [3441.01764566],
       [4721.56123888],
       [4024.21695878],
       [3382.04502   ],
       [3737.00309184],
       [3514.67126422],
       [3476.20744555],
       [3378.31515437],
       [5554.7131803 ],
       [3578.56060893],
       [4102.24254257],
       [3923.23054465],
       [4113.4431545 ],
       [4396.16456616],
       [3458.44052825],
       [3539.75230543],
       [3388.75726727],
       [5408.83319697],
       [4240.91989667],
       [5948.04475847],
       [4688.86243867],
       [5462.97723483],
       [5569.40907063],
       [5584.73156324],
       [4290.21258529],
       [3406.02557957],
       [4051.72965017],
       [4214.06283803],
       [3391.01367973],
       [3523.88920303],
       [4652.82489452],
       [3361.11357249],
       [3497.29005187],
       [3286.81955108],
       [3464.77601115],
       [3979.26395218],
       [4956.34137466],
       [3483.00436785],
       [5657.824

In [ ]:
pd.DataFrame({'body_mass_g' : y_pred[:,0]}).to_csv('123.csv', index=False)

# 실습 3. Census Income

## 데이터 수집

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
X_train = pd.read_csv("/content/yemoonsaBigdata/datasets/Part2/census_X_train.csv")
y_train = pd.read_csv("/content/yemoonsaBigdata/datasets/Part2/census_y_train.csv")
X_test = pd.read_csv("/content/yemoonsaBigdata/datasets/Part2/census_X_test.csv")

## 데이터 전처리

In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30162 entries, 0 to 30161
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             30162 non-null  int64 
 1   workclass       30162 non-null  object
 2   education_num   30162 non-null  int64 
 3   marital_status  30162 non-null  object
 4   occupation      30162 non-null  object
 5   relationship    30162 non-null  object
 6   race            30162 non-null  object
 7   sex             30162 non-null  object
 8   capital_gain    30162 non-null  int64 
 9   capital_loss    30162 non-null  int64 
 10  hours_per_week  30162 non-null  int64 
 11  native_country  30162 non-null  object
dtypes: int64(5), object(7)
memory usage: 2.8+ MB


In [ ]:
X_train.describe()

# capital_gain, capital_loss가 한 쪽으로 크게 치우쳐져 있음.

,age,education_num,capital_gain,capital_loss,hours_per_week
count,30162.000000,30162.000000,30162.000000,30162.000000,30162.000000
mean,38.437902,10.121312,1092.007858,88.372489,40.931238
std,13.134665,2.549995,7406.346497,404.298370,11.979984
min,17.000000,1.000000,0.000000,0.000000,1.000000
25%,28.000000,9.000000,0.000000,0.000000,40.000000
50%,37.000000,10.000000,0.000000,0.000000,40.000000
75%,47.000000,13.000000,0.000000,0.000000,45.000000
max,90.000000,16.000000,99999.000000,4356.000000,99.000000


In [ ]:
X_train['capital_gain'].quantile([q/20 for q in range(15, 21)])

0.75        0.0
0.80        0.0
0.85        0.0
0.90        0.0
0.95     5013.0
1.00    99999.0
Name: capital_gain, dtype: float64

In [ ]:
X_train['capital_loss'].quantile([q/20 for q in range(15, 21)])

0.75       0.0
0.80       0.0
0.85       0.0
0.90       0.0
0.95       0.0
1.00    4356.0
Name: capital_loss, dtype: float64

In [ ]:
# 범주형 변수로 변환

X_train['capital_gain_yn'] = np.where(X_train['capital_gain']>0, 1, 0)
X_train['capital_loss_yn'] = np.where(X_train['capital_loss']>0, 1, 0)

X_test['capital_gain_yn'] = np.where(X_test['capital_gain']>0, 1, 0)
X_test['capital_loss_yn'] = np.where(X_test['capital_loss']>0, 1, 0)

In [ ]:
# 변수 형태별 리스트 저장
COL_DEL = []
COL_NUM = ['age', 'education_num', 'hours_per_week', 'capital_gain', 'capital_loss']
COL_CAT = ['workclass', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'native_country', 'capital_gain_yn', 'capital_loss_yn']
COL_Y = ['target']

X_train = X_train.drop(COL_DEL, axis=1)
X_test = X_test.drop(COL_DEL, axis=1)

In [ ]:
# 수치형 변수 확인

train_df = pd.concat([X_train, y_train], axis=1)

for _col in COL_NUM:
  print('-'*80)
  print(_col)
  print(train_df.groupby(COL_Y)[_col].describe(), end='\n\n')

--------------------------------------------------------------------------------
age
          count      mean        std   min   25%   50%   75%   max
target                                                            
0       22654.0  36.60806  13.464631  17.0  26.0  34.0  45.0  90.0
1        7508.0  43.95911  10.269633  19.0  36.0  43.0  51.0  90.0

--------------------------------------------------------------------------------
education_num
          count       mean       std  min   25%   50%   75%   max
target                                                           
0       22654.0   9.629116  2.413596  1.0   9.0   9.0  10.0  16.0
1        7508.0  11.606420  2.368423  2.0  10.0  12.0  13.0  16.0

--------------------------------------------------------------------------------
hours_per_week
          count       mean        std  min   25%   50%   75%   max
target                                                            
0       22654.0  39.348592  11.950774  1.0  38.0  40.0  

In [ ]:
# 범주형 변수 확인

for _col in COL_CAT:
  print(train_df.groupby(_col, as_index=False)[COL_Y].mean().sort_values(by=COL_Y, ascending=False), end='\n\n')

# cardinality: 범주형 변수의 값의 가짓 수, 클수록 모델 성능에 악영향, 보통 100개 넘으면 높다고 함.
# cardinality 발생 시: 해당 변수 제거 or 일부 범주 합치기 or 타겟 인코딩

          workclass    target
3      Self-emp-inc  0.558659
0       Federal-gov  0.387063
1         Local-gov  0.294630
4  Self-emp-not-inc  0.285714
5         State-gov  0.268960
2           Private  0.218792
6       Without-pay  0.000000

          marital_status    target
1      Married-AF-spouse  0.476190
2     Married-civ-spouse  0.454959
0               Divorced  0.107262
6                Widowed  0.096735
3  Married-spouse-absent  0.083784
5              Separated  0.070288
4          Never-married  0.048324

           occupation    target
3     Exec-managerial  0.485220
9      Prof-specialty  0.448489
10    Protective-serv  0.326087
12       Tech-support  0.304825
11              Sales  0.270647
2        Craft-repair  0.225310
13   Transport-moving  0.202926
0        Adm-clerical  0.133835
6   Machine-op-inspct  0.124619
4     Farming-fishing  0.116279
1        Armed-Forces  0.111111
5   Handlers-cleaners  0.061481
7       Other-service  0.041096
8     Priv-house-serv  0.00699

In [ ]:
# label incoding (범주형에 한해서만), 정수형으로 바꾼 것이므로 선형 모델에는 적합하지 않음.

from sklearn.preprocessing import LabelEncoder

X = pd.concat([X_train, X_test])
for _col in COL_CAT:
  le = LabelEncoder()
  le.fit(X_train[_col])
  X_train[_col] = le.transform(X_train[_col])
  X_test[_col] = le.transform(X_test[_col])

## 모델링

In [ ]:
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.3, stratify=y_train)

In [ ]:
# scaling (수치형에 한해서만), validation 제외한 순수한 training data로만 scaler를 fitting 해야 함.
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_tr[COL_NUM] = scaler.fit_transform(X_tr[COL_NUM])
X_val[COL_NUM] = scaler.transform(X_val[COL_NUM])
X_test[COL_NUM] = scaler.transform(X_test[COL_NUM])

In [ ]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

model_rf = RandomForestClassifier()
model_rf.fit(X_tr, y_tr.values.ravel())

RandomForestClassifier()

In [ ]:
# XGBoost
from xgboost import XGBClassifier

model_xgb1 = XGBClassifier()
model_xgb1.fit(X_tr, y_tr.values.ravel())

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [ ]:
# XGBoost (overfitting 방지)
from xgboost import XGBClassifier

model_xgb2 = XGBClassifier(n_estimators=1000, learning_rate=0.1, max_depth=10)
model_xgb2.fit(X_tr, y_tr.values.ravel(), early_stopping_rounds=50, eval_metric='auc', eval_set=[(X_val, y_val)], verbose=10)

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-auc:0.90604
[10]	validation_0-auc:0.91544
[20]	validation_0-auc:0.91873
[30]	validation_0-auc:0.92218
[40]	validation_0-auc:0.92408
[50]	validation_0-auc:0.92478
[60]	validation_0-auc:0.92539
[70]	validation_0-auc:0.92558
[80]	validation_0-auc:0.92581
[90]	validation_0-auc:0.92577
[100]	validation_0-auc:0.92567
[110]	validation_0-auc:0.92544
[120]	validation_0-auc:0.92522
[130]	validation_0-auc:0.92490
[132]	validation_0-auc:0.92489


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1000, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

## 평가

In [ ]:
from sklearn.metrics import roc_auc_score

y_pred_rf = model_rf.predict_proba(X_val)
y_pred_xgb1 = model_xgb1.predict_proba(X_val)

score_rf = roc_auc_score(y_val, y_pred_rf[:, 1])
score_xgb1 = roc_auc_score(y_val, y_pred_xgb1[:, 1])

print(score_rf, score_xgb1)

0.8942651404822574 0.9252077044752007


## 모델 성능 향상 작업 (hyperparameter tuning)

In [ ]:
# capital_gain_yn, capital_loss_yn 등의 범주화가 모델 성능 향상에 도움을 주었는지 확인

pd.DataFrame({'feature' : X_tr.columns, 'fi_rf' : model_rf.feature_importances_, 'fi_xgb' : model_xgb1.feature_importances_})

,feature,fi_rf,fi_xgb
0,age,0.221642,0.031758
1,workclass,0.049702,0.020215
2,education_num,0.138069,0.106748
3,marital_status,0.068152,0.186587
4,occupation,0.089826,0.033688
5,relationship,0.109732,0.316754
6,race,0.017913,0.015819
7,sex,0.012896,0.033650
8,capital_gain,0.101409,0.156429
9,capital_loss,0.030533,0.059623


In [ ]:
# capital_gain_yn, capital_loss_yn이 capital_gain, capital_loss보다 importance가 낮으므로 capital_gain_yn, capital_loss_yn 삭제

COL_DEL = ['capital_gain_yn', 'capital_loss_yn']

X_tr = X_tr.drop(COL_DEL, axis=1)
X_val = X_val.drop(COL_DEL, axis=1)
X_test = X_test.drop(COL_DEL, axis=1)

In [ ]:
# Random Forest hyperparameter tuning

from sklearn.model_selection import GridSearchCV

grid_params = {
    'n_estimators' : [50, 100, 200],
    'max_depth' : [5, 10, 15],
    'min_samples_split' : [2, 5, 10],
    'min_samples_leaf' : [1, 2, 4]
}

rf_cv = GridSearchCV(estimator=model_rf, param_grid=grid_params, cv=5)
rf_cv.fit(X_tr, y_tr.values.ravel())

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [5, 10, 15],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 200]})

In [ ]:
# best hyperparameter

rf_cv.best_params_

{'max_depth': 15,
 'min_samples_leaf': 1,
 'min_samples_split': 5,
 'n_estimators': 200}

In [ ]:
# 모델 재학습

model_rf2 = RandomForestClassifier(n_estimators=50, max_depth=15, min_samples_leaf=1, min_samples_split=5)
model_rf2.fit(X_tr, y_tr.values.ravel())

y_pred_rf2 = model_rf2.predict_proba(X_val)
score_rf2 = roc_auc_score(y_val, y_pred_rf2[:, 1])
score_rf2


0.9179005482776201

In [ ]:
# XGBoost hyperparameter tuning

from sklearn.model_selection import GridSearchCV

grid_params = {
    'max_depth' : [3, 5, 7, 10],
    'min_child_weight' : [1, 2],
    'colsample_bytree' : [0.6, 0.8],
    'subsample' : [0.6, 0.8]
}

xgb_cv = GridSearchCV(estimator=model_xgb1, param_grid=grid_params, cv=5)
xgb_cv.fit(X_tr, y_tr.values.ravel())

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             param_grid={'colsample_bytree': [0.6, 0.8],
                         'max_depth': [3, 5, 7, 10], 'min_child_weight': [1, 2],
                         'subsample': [0.6, 0.8]})

In [ ]:
# best hyperparameter

xgb_cv.best_params_

{'colsample_bytree': 0.6,
 'max_depth': 3,
 'min_child_weight': 2,
 'subsample': 0.8}

In [ ]:
# 모델 재학습

params = {'colsample_bytree' : 0.6, 'max_depth' : 7, 'min_child_weight' : 1, 'subsample' : 0.8}

model_xgb3 = XGBClassifier(n_estimators=1000, learning_rate=0.05)
model_xgb3.set_params(**params)

model_xgb3.fit(X_tr, y_tr, early_stopping_rounds=50, eval_metric='auc', eval_set=[(X_val, y_val)], verbose=10)

model_xgb3.best_score

[0]	validation_0-auc:0.85616
[10]	validation_0-auc:0.91335


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[20]	validation_0-auc:0.91716
[30]	validation_0-auc:0.91889
[40]	validation_0-auc:0.91994
[50]	validation_0-auc:0.92141
[60]	validation_0-auc:0.92260
[70]	validation_0-auc:0.92342
[80]	validation_0-auc:0.92446
[90]	validation_0-auc:0.92512
[100]	validation_0-auc:0.92602
[110]	validation_0-auc:0.92663
[120]	validation_0-auc:0.92733
[130]	validation_0-auc:0.92763
[140]	validation_0-auc:0.92786
[150]	validation_0-auc:0.92799
[160]	validation_0-auc:0.92798
[170]	validation_0-auc:0.92810
[180]	validation_0-auc:0.92814
[190]	validation_0-auc:0.92806
[200]	validation_0-auc:0.92802
[210]	validation_0-auc:0.92796
[220]	validation_0-auc:0.92812
[225]	validation_0-auc:0.92814


0.9281993727773015

## 예측

In [ ]:
pred = model_xgb3.predict_proba(X_test)[:, 1]
pd.DataFrame({'index' : X_test.index, 'target' : pred}).to_csv('123.csv', index=False)